# 🛋️ Furniture Detection with YOLOv8

## 1. Environment Setup & Installation
**Goal:** Prepare the computer (Google Colab) to run the code.

We need to install specific software libraries:
* **`ultralytics`**: The main library for YOLOv8 (the AI model we are using).
* **`awscli`**: A tool to download images directly from the OpenImages database (hosted on Amazon AWS).
* **`urllib3`**: A helper library for internet connections (we downgrade it to avoid version conflicts).

> **Note:** We verify the installation at the end to make sure the GPU (Graphics Card) is active, which makes training much faster.

In [ ]:
# --- 1. Install Helper Libraries ---
# We use '%%capture' to hide the long installation logs and keep the notebook clean.
%%capture
!pip install cython pyyaml requests ultralytics

# --- 2. AWS CLI & Connection Fixes ---
# OpenImages dataset is hosted on AWS. We need the AWS Command Line Interface (CLI) to download it.
# Sometimes Colab has conflicting versions, so we clean up and reinstall.
!pip uninstall -y awscli botocore urllib3
!sudo apt-get remove -y awscli
!pip install --upgrade 'urllib3<2'  # Fix for a common connection error
!pip install awscli

# --- 3. Verify Installation ---
import torch
print("Setup Complete.")
print(f"CUDA (GPU) available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: You are running on CPU. Go to Runtime > Change runtime type > Hardware accelerator > T4 GPU.")

## 2. Mount Drive & Configuration
**Goal:** Connect Google Drive to save our data and model permanently.

Since Google Colab deletes files after the session ends, we save everything to Google Drive.

We also define **Constants**:  
- **`TARGET_CLASSES_LIST`**: The specific furniture items we want to detect.  
- **`DIRS`**: A dictionary that stores where images, labels, and CSV files are located.

### 📁 Required Folder Structure (Tree)

Your Google Drive should contain the following directory tree:

MyDrive/

    └── furniture-recommender/
        └── yolo-train/
            ├── class-descriptions-boxable.csv
            ├── train-annotations-bbox.csv
            ├── test-annotations-bbox.csv

After running the setup, the images and lables will downloaded properly

In [ ]:
import os
import multiprocessing
import random
import pandas as pd
import shutil
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# --- Configuration ---

# Define the main working directory in Google Drive
BASE_DIR = "/content/drive/MyDrive/furniture-recommender/yolo-train"

# Create the directory if it doesn't exist
os.makedirs(BASE_DIR, exist_ok=True)
os.chdir(BASE_DIR)

# The specific objects we want the AI to learn
TARGET_CLASSES_LIST = ['Bed', 'Cabinetry', 'Chair', 'Couch', 'Lamp', 'Table']

# Define file paths for organization
# YOLO requires a specific structure: images/ and labels/ folders.
DIRS = {
    "train": {
        "root": os.path.join(BASE_DIR, "train"),
        "images": os.path.join(BASE_DIR, "train", "images"),
        "labels": os.path.join(BASE_DIR, "train", "labels"),
        "csv": os.path.join(BASE_DIR, "train-annotations-bbox.csv") # Requires OpenImages csv
    },
    "test": {
        "root": os.path.join(BASE_DIR, "test"),
        "images": os.path.join(BASE_DIR, "test", "images"),
        "labels": os.path.join(BASE_DIR, "test", "labels"),
        "csv": os.path.join(BASE_DIR, "test-annotations-bbox.csv") # Requires OpenImages csv
    }
}

# Path to the file that translates Class IDs (e.g., "/m/01xy") to Names (e.g., "Bed")
CLASS_DESC_FILE = os.path.join(BASE_DIR, "class-descriptions-boxable.csv")

# Create the necessary directories automatically
for split in DIRS:
    os.makedirs(DIRS[split]["images"], exist_ok=True)
    os.makedirs(DIRS[split]["labels"], exist_ok=True)

print(f"✅ Configuration loaded. Working directory: {BASE_DIR}")

## 3. Data Download & Label Preparation
**Goal:** Build the dataset.

This step is the most complex part of the preparation. It does two things:
1.  **Download Images:** It looks at the CSV files from OpenImages, finds images that contain our target furniture (Bed, Chair, etc.), and downloads them from AWS.
2.  **Convert Labels:** OpenImages uses a different format for coordinates than YOLO.
    * *OpenImages:* `XMin, XMax, YMin, YMax`
    * *YOLO:* `Center_X, Center_Y, Width, Height` (normalized between 0 and 1).

**Optimization:** We use `ThreadPoolExecutor` to download multiple images at once, making it much faster.

In [ ]:
import csv
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

# --- Helper: Map Class Names to IDs ---
def get_class_id_map(class_desc_file):
    """
    Reads the CSV file that links computer codes (e.g., /m/03ssj5) to human names (e.g., Bed).
    Returns a dictionary: {'Bed': '/m/03ssj5', ...}
    """
    if not os.path.exists(class_desc_file):
        raise FileNotFoundError(f"CRITICAL: Class description file missing at {class_desc_file}")

    print("Loading class descriptions...")
    df_desc = pd.read_csv(class_desc_file, header=None, names=['ClassId', 'LabelName'])
    df_desc['LabelName'] = df_desc['LabelName'].str.replace('_', ' ') # Fix names like 'Office_Chair' -> 'Office Chair'
    return df_desc.set_index('LabelName')['ClassId'].to_dict()

# --- Helper: Download Single Image ---
def download_image(aws_cmd):
    """Runs the terminal command to download a file from AWS S3."""
    os.system(aws_cmd)

# --- Step 1: Download Images ---
def download_dataset(dataset_type, classes, max_images_per_class=1200):
    """
    Downloads images for the requested classes.
    args:
        dataset_type: 'train' or 'test'
        classes: List of furniture names
        max_images_per_class: Limit to avoid downloading too much data
    """
    print(f"\n--- ⬇️ Starting download for: {dataset_type} ---")

    images_dir = DIRS[dataset_type]['images']
    annotation_file = DIRS[dataset_type]['csv']

    if not os.path.exists(annotation_file):
        print(f"⚠️ Skipping {dataset_type}: Annotation CSV not found at {annotation_file}")
        return

    # 1. Get Class IDs (e.g., Bed -> /m/03ssj5)
    class_name_to_id = get_class_id_map(CLASS_DESC_FILE)

    # 2. Load Annotations CSV
    print(f"Loading annotations from {os.path.basename(annotation_file)}...")
    # We only read ImageID and LabelName to save memory
    df_ann = pd.read_csv(annotation_file, usecols=['ImageID', 'LabelName'])

    commands = []

    # 3. Filter images for each class
    for class_name in classes:
        # Ensure format matches CSV (remove underscores)
        normalized_name = class_name.replace("_", " ")
        class_id = class_name_to_id.get(normalized_name)

        if not class_id:
            print(f"  ⚠️ Class '{class_name}' not found in OpenImages data.")
            continue

        # Filter rows matching this class
        df_class = df_ann[df_ann['LabelName'] == class_id]
        image_ids = df_class['ImageID'].tolist()

        # Randomly sample if we have too many images
        if len(image_ids) > max_images_per_class:
            image_ids = random.sample(image_ids, max_images_per_class)

        print(f"  > {class_name}: Found {len(image_ids)} images.")

        # Prepare download commands
        for image_id in image_ids:
            target_path = os.path.join(images_dir, f"{image_id}.jpg")
            # Only download if we don't have it yet
            if not os.path.exists(target_path):
                # AWS S3 command for OpenImages (no sign request needed = public)
                cmd = f'aws s3 --no-sign-request --only-show-errors cp s3://open-images-dataset/{dataset_type}/{image_id}.jpg {target_path}'
                commands.append(cmd)

    # 4. Execute Downloads (Multi-threaded)
    # Remove duplicates (an image might have both a Bed and a Lamp)
    commands = list(set(commands))
    print(f"  > Downloading {len(commands)} new images...")

    if commands:
        # Use 4x CPU count for threads (IO bound operation)
        workers = multiprocessing.cpu_count() * 4
        with ThreadPoolExecutor(max_workers=workers) as executor:
            list(tqdm(executor.map(download_image, commands), total=len(commands)))
    else:
        print("  > All images already downloaded.")

# --- Step 2: Create YOLO Labels ---
def prepare_yolo_labels(split_name):
    """
    Converts OpenImages CSV data into YOLO .txt files.
    YOLO format: class_id center_x center_y width height
    """
    print(f"\n--- 🏷️ Generating Labels for: {split_name} ---")
    annotations_path = DIRS[split_name]['csv']
    output_dir = DIRS[split_name]['labels']

    if not os.path.exists(annotations_path):
        return

    # 1. Load Class Names mapping
    names_df = pd.read_csv(CLASS_DESC_FILE, header=None, names=['LabelName', 'HumanName'])

    # 2. Load Annotations (Bounding Boxes)
    df = pd.read_csv(annotations_path)

    # 3. Merge to get readable names (e.g., 'Bed' instead of '/m/03ssj5')
    df = df.merge(names_df, on='LabelName', how='left')

    # Filter only our target furniture classes
    df = df[df['HumanName'].isin(TARGET_CLASSES_LIST)]

    # 4. Map names to YOLO IDs (0, 1, 2, 3...)
    # It's crucial to sort the list so ID 0 is always the same class
    TARGET_CLASSES_LIST.sort()
    class_map = {name: idx for idx, name in enumerate(TARGET_CLASSES_LIST)}
    print(f"  > Class Map: {class_map}")

    # 5. Calculate YOLO Coordinates
    # OpenImages gives min/max. YOLO needs center + width/height relative to image size (0-1).
    df['width'] = df['XMax'] - df['XMin']
    df['height'] = df['YMax'] - df['YMin']
    df['x_center'] = df['XMin'] + (df['width'] / 2)
    df['y_center'] = df['YMin'] + (df['height'] / 2)
    df['class_id'] = df['HumanName'].map(class_map)

    # 6. Save .txt files
    # Group data by image so we write one file per image
    grouped = df.groupby('ImageID')

    count = 0
    for image_id, group in tqdm(grouped, desc="Writing .txt files"):
        img_path = os.path.join(DIRS[split_name]['images'], f"{image_id}.jpg")

        # Only create a label file if the image actually exists (was downloaded)
        if os.path.exists(img_path):
            txt_filename = os.path.join(output_dir, f"{image_id}.txt")
            with open(txt_filename, 'w') as f:
                for _, row in group.iterrows():
                    # Write line: class_id x y w h
                    line = f"{int(row['class_id'])} {row['x_center']:.6f} {row['y_center']:.6f} {row['width']:.6f} {row['height']:.6f}\n"
                    f.write(line)
            count += 1
    print(f"  > Created {count} label files.")

# --- EXECUTE PIPELINE ---
# 1. Download Train Data
download_dataset("train", TARGET_CLASSES_LIST)
prepare_yolo_labels("train")

# 2. Download Test Data
download_dataset("test", TARGET_CLASSES_LIST)
prepare_yolo_labels("test")

## 4. Create Dataset Config (data.yaml)
**Goal:** Tell YOLO where the data is.

YOLO training requires a `.yaml` file that specifies:
1.  Path to **Train** images.
2.  Path to **Validation/Test** images.
3.  Number of classes (`nc`).
4.  List of class names (`names`).

In [ ]:
import yaml

# Ensure the list is sorted same as in label generation
TARGET_CLASSES_LIST.sort()

yaml_content = {
    'train': DIRS['train']['images'],
    'val': DIRS['test']['images'], # We use test set for validation here
    'nc': len(TARGET_CLASSES_LIST),
    'names': TARGET_CLASSES_LIST
}

yaml_path = os.path.join(BASE_DIR, 'data.yaml')

with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=None)

print(f"✅ data.yaml created at: {yaml_path}")

## 5. Train YOLOv8 Model (Auto-Resume Support)
**Goal:** Teach the AI to recognize furniture.

We load the base model (`yolov8n.pt` - "Nano", the fastest and smallest version).
Then we start training for 200 epochs (cycles).

**Auto-Resume Logic:**
Training can take hours. If Google Colab disconnects, we don't want to start from zero.
The code below checks if a previous training run exists.
* If `last.pt` exists: It loads it and sets `resume=True`.
* If not: It starts a fresh training session.

In [ ]:
from ultralytics import YOLO

# Project Name (Folder name where results are saved)
PROJECT_NAME = 'furniture_detector_model'
RUNS_DIR = os.path.join(BASE_DIR, 'runs', 'detect')
LAST_CHECKPOINT = os.path.join(RUNS_DIR, PROJECT_NAME, 'weights', 'last.pt')

print(f"Checking for existing checkpoint at: {LAST_CHECKPOINT}")

if os.path.exists(LAST_CHECKPOINT):
    print("🔄 Found incomplete training. Resuming from last checkpoint...")
    # Load the partially trained model
    model = YOLO(LAST_CHECKPOINT)
    resume_training = True
else:
    print("✨ No previous checkpoint found. Starting fresh training...")
    # Load a pre-trained generic model (Transfer Learning)
    model = YOLO('yolov8n.pt')
    resume_training = False

# Start Training
# If resuming, we don't need to specify args again, YOLO reads them from the file.
if resume_training:
    results = model.train(resume=True)
else:
    results = model.train(
        data=os.path.join(BASE_DIR, 'data.yaml'),
        epochs=200,          # How many times to go over the data
        imgsz=640,           # Image resolution
        batch=32,            # How many images to process at once
        device=0,            # Use GPU (0)
        project=os.path.join(BASE_DIR, 'runs/detect'),
        name=PROJECT_NAME,
        exist_ok=True,       # Allow overwriting folder if not resuming
        patience=50          # Stop early if no improvement for 50 epochs
    )

print("✅ Training Complete.")

## 6. Test on Random Image
**Goal:** See the results with our own eyes.

We pick a random image from the test set, run the trained model, and visualize the detected boxes.
We also perform cleanup to avoid filling the Google Drive with temporary crop files.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from glob import glob

# Path to the best model saved during training
best_model_path = os.path.join(RUNS_DIR, PROJECT_NAME, 'weights', 'best.pt')
inference_output = os.path.join(BASE_DIR, 'inference_results')

if not os.path.exists(best_model_path):
    print(f"❌ Error: Model not found at {best_model_path}. Did training finish?")
else:
    # Load the trained model
    model = YOLO(best_model_path)

    # Find test images
    test_images_dir = DIRS['test']['images']
    all_test_images = glob(os.path.join(test_images_dir, "*.jpg"))

    if not all_test_images:
        print("No images found in test directory.")
    else:
        # Pick one random image
        random_image = random.choice(all_test_images)
        print(f"Testing on image: {os.path.basename(random_image)}")

        # Run Inference (Prediction)
        # save_crop=True creates small images of detected objects
        results = model.predict(
            source=random_image,
            save=True,
            save_crop=True,
            project=inference_output,
            name='prediction',
            exist_ok=True, # Overwrite previous prediction folder
            conf=0.25      # Minimum confidence (25%) to show a box
        )

        # --- Visualization ---
        # Plot the main image with boxes
        plt.figure(figsize=(10, 6))
        # results[0].plot() returns a BGR array, convert to RGB for Matplotlib
        res_plotted = results[0].plot()
        plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
        plt.title(f"Detections: {os.path.basename(random_image)}")
        plt.axis('off')
        plt.show()

        # --- Clean Up ---
        # We delete the 'inference_results' folder to keep Drive clean
        if os.path.exists(inference_output):
             shutil.rmtree(inference_output)
             print(f"🧹 Cleanup: Deleted temporary folder {inference_output}")